In [59]:
import numpy as np
import pandas as pd
import FinanceDataReader as fdr
import yfinance as yf
import datetime
import requests # Requests to grab source code from Wiki page
from bs4 import BeautifulSoup

In [48]:
df_krx = fdr.StockListing('KRX')

In [49]:
df_krx = df_krx[(df_krx.Market == 'KOSPI')]
df_krx.reset_index(inplace = True, drop=True)

In [50]:
df_krx200 = df_krx.iloc[0:200,:]
tickers_krx200 = list(df_krx200.Code)

In [55]:
tickers_krx200 = [i + '.KS' for i in tickers_krx200]

In [57]:
start = datetime.datetime(2007 ,1 ,1)
end = datetime.datetime(2022, 12, 31)
data = yf.download(tickers_krx200, start = start, end = end)

[*********************100%***********************]  200 of 200 completed


In [60]:
data.index = pd.to_datetime(data.index)
data.index = data.index.strftime('%Y-%m-%d')

In [63]:
df = data.stack().reset_index().rename(index = str,  # .stack() moves the 2nd row of the columns to the rows,
                                       columns = {
                                           "level_1":"Symbol"
                                       }).sort_values(['Symbol', 'Date']) #  .reset_index() collapses the two rows of columns into a single one
df.set_index('Date', inplace = True)

In [72]:
# df.to_csv(r"/Users/skim/Desktop/Code/Quant/data/kospi200.csv")
df_stocks = pd.read_csv(r"/Users/skim/Desktop/Code/Quant/data/snp500.csv")

In [66]:
ticker_names = np.unique(df['Symbol'])
df_dividends = pd.DataFrame(0, columns = ticker_names, index = data.index)

for i in ticker_names[0:5]:
    temp_ticker = yf.Ticker(str(i))
    temp_div_data = yf.Ticker(str(i)).history(start = start, end = end)['Dividends']
    temp_div_data.index = pd.to_datetime(temp_div_data.index)
    temp_div_data.index = temp_div_data.index.strftime('%Y-%m-%d')
    df_dividends[i] = temp_div_data

In [73]:
# df_dividends.to_csv(r"/Users/skim/Desktop/Code/Quant/data/kospi200_dividends.csv")
